In [1]:
import pandas as pd
from pathlib import Path
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
parent_dir = os.path.dirname(r"C:\Users\phung\Documents\DATA 4419\Instacart-product-recommendation\notebooks")
print(os.getcwd())

C:\Users\phung\Documents\DATA 4419\Instacart-product-recommendation\notebooks


In [2]:
project_root = Path.cwd().parent  
data_path = project_root / "csv_files"
RAW = Path("../data/raw/")
PROCESSED = Path("data/processed")
PROCESSED.mkdir(parents = True, exist_ok = True)


Splits the orders DataFrame into three separate DataFrames based on the 'eval_set' column:
    * orders_prior: all rows where eval_set == 'prior'
    * orders_train: all rows where eval_set == 'train'
    * orders_test: all rows where eval_set == 'test'
    Only the key columns needed for analysis are kept in each output DataFrame.
    Returns: orders_prior, orders_train, orders_test (DataFrames)

In [15]:
def split_orders_by_eval_set(orders): 
    columns = [
        "order_id", "user_id", "order_number",
        "order_dow", "order_hour_of_day", "days_since_prior_order"]
    orders_prior = orders[orders["eval_set"] == "prior"][columns]
    orders_train = orders[orders["eval_set"] == "train"][columns]
    orders_test = orders[orders["eval_set"] == "test"][columns]
    return orders_prior, orders_train, orders_test

Loads all six Instacart CSVs from the data/raw/ directory into pandas DataFrames with memoery efficient dtypes.

Performs initial safety checks and prints summary statistics for the orders DataFrame.

Returns:
    A dictionary containing all loaded DataFrames: orders, op_prior, op_train, products, aisles, departments.

In [41]:
def load_all_data(raw_path):
    op_dtype = {"order_id": "int32",
                "product_id": "int32",
                "add_to_cart_order": "int16",
                "reordered": "int8"}
    
    orders = pd.read_csv(raw_path / "orders.csv",
        dtype = {"order_id":"int32",
            "user_id":"int32",
            "order_number":"int16",
            "order_dow":"int8",
            "order_hour_of_day":"int8"
        },
        keep_default_na = True,
    )
    orders["eval_set"] = orders["eval_set"].astype(str).str.lower()

    if "days_since_prior_order" in orders.columns:
        orders["days_since_prior_order"] = pd.to_numeric(
        orders["days_since_prior_order"], errors = "coerce").astype("float32")

    op_prior = pd.read_csv(raw_path / "order_products__prior.csv", dtype=op_dtype)
    op_train = pd.read_csv(raw_path / "order_products__train.csv", dtype=op_dtype)

    products = pd.read_csv(raw_path / "products.csv")
    aisles = pd.read_csv(raw_path / "aisles.csv")
    departments = pd.read_csv(raw_path / "departments.csv")

    # return all necessary DataFrames from the function
    return {
        "orders": orders,
        "op_prior": op_prior,
        "op_train": op_train,
        "products": products,
        "aisles": aisles,
        "departments": departments
    }

Splits the orders DataFrame into prior, train, and test sets, then merges prior orders with order_products__prior.csv
to create a line-item table (prior) with user and order context for each product in prior orders.

In [17]:
def build_prior_line_items(dfs):
    orders = dfs["orders"]
    op_prior = dfs["op_prior"]

    # Split the orders DataFrame into prior, train, and test sets using the helper function
    orders_prior, orders_train, orders_test = split_orders_by_eval_set(orders)
    
    prior = op_prior.merge(
        orders_prior,
        on = "order_id",
        how = "inner")
    return prior, orders_prior, orders_train, orders_test


In [43]:
all_data = load_all_data(RAW)
orders = all_data["orders"]
op_train = all_data["op_train"]
prior, orders_prior, orders_train, orders_test = build_prior_line_items(all_data)

In [21]:
orders_prior.isnull().sum()
orders_prior.duplicated().sum()
orders_prior.dtypes
orders_prior['order_id'].nunique()
len(orders_prior)
orders_prior['order_number'].min()
orders_prior['order_number'].max()

orders_prior.shape
orders_train.shape
orders_test.shape
prior.shape

order_id                       0
user_id                        0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

np.int64(0)

order_id                    int32
user_id                     int32
order_number                int16
order_dow                    int8
order_hour_of_day            int8
days_since_prior_order    float32
dtype: object

3214874

3214874

...

np.int16(99)

(3214874, 6)

(131209, 6)

(75000, 6)

(32434489, 9)

Main workflow for the script. Loads all data, prints summary, builds core tables, and optionally saves processed results.
    Args: save_processed (bool): If True, saves the processed DataFrames to disk.

In [29]:
def main(save_processed = False): 
    dfs = load_all_data(RAW)

    for k, v in dfs.items():
        print(f"{k}: shape = {v.shape}, columns = {v.columns.tolist()}") # k is key, v is DataFrame
    # merge prior orders with prior products
    prior, orders_prior, orders_train, orders_test = build_prior_line_items(dfs)

    # Get op_train from loaded data
    op_train = dfs["op_train"] 
    if save_processed:
        prior.to_csv(PROCESSED / "prior_line_items_merged.csv", index = False)
        print(f"Processed data saved to {PROCESSED}")

main(save_processed = False)

orders: shape = (3421083, 7), columns = ['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']
op_prior: shape = (32434489, 4), columns = ['order_id', 'product_id', 'add_to_cart_order', 'reordered']
op_train: shape = (1384617, 4), columns = ['order_id', 'product_id', 'add_to_cart_order', 'reordered']
products: shape = (49688, 4), columns = ['product_id', 'product_name', 'aisle_id', 'department_id']
aisles: shape = (134, 2), columns = ['aisle_id', 'aisle']
departments: shape = (21, 2), columns = ['department_id', 'department']


In [22]:
# Build user features from prior only
user_grp = prior.groupby("user_id")
user_features = pd.DataFrame({
    # User ID
    "user_id": user_grp["user_id"].first(),
    # Total number of orders for each user
    "u_total_orders": user_grp["order_number"].max(),
    # Total number of items ordered by each user
    "u_total_items": user_grp["product_id"].count(),
    # Number of distinct products ordered by each user
    "u_distinct_products": user_grp["product_id"].nunique(),
    # Average basket size: items per unique order
    "u_avg_basket_size": user_grp["order_id"].count() / user_grp["order_id"].nunique(),
    # Reorder ratio: mean of reordered flag (0 if not present)
    "u_reorder_ratio": user_grp["reordered"].mean().fillna(0),
    # Mean days between orders (ignoring NaNs)
    "u_mean_days_between": user_grp["days_since_prior_order"].mean(),
    # Std dev of days between orders (ignoring NaNs)
    "u_std_days_between": user_grp["days_since_prior_order"].std()
    })

user_features = user_features.reset_index(drop = True)
user_features.to_csv(PROCESSED / "ql_user_features.csv", index = False)
print("Saved user features to data/processed/ql_user_features.csv")
user_features.head()

Saved user features to data/processed/ql_user_features.csv
   user_id  u_total_orders  u_total_items  u_distinct_products  \
0        1              10             59                   18   
1        2              14            195                  102   
2        3              12             88                   33   
3        4               5             18                   17   
4        5               4             37                   23   

   u_avg_basket_size  u_reorder_ratio  u_mean_days_between  u_std_days_between  
0           5.900000         0.694915            20.259260            9.304462  
1          13.928571         0.476923            15.967033            9.119768  
2           7.333333         0.625000            11.487180            4.869048  
3           3.600000         0.055556            15.357142            8.580901  
4           9.250000         0.378378            14.500000            4.263801  


In [58]:
# Build product features from prior only
product_grp = prior.groupby("product_id")
product_features = pd.DataFrame({
    "product_id": product_grp["product_id"].first(),
    "p_total_purchases": product_grp["order_id"].count(),
    "p_distinct_users": product_grp["user_id"].nunique(),
    "p_avg_add_to_cart": product_grp["add_to_cart_order"].mean(),
    "p_reorder_prob": product_grp["reordered"].mean().fillna(0)
    })

# reset index so that product_id is column, not index
product_features = product_features.reset_index(drop = True)

product_features.to_csv(PROCESSED / "ql_product_features.csv", index=False)
print("Saved product features to data/processed/ql_product_features.csv")
product_features.head()

Saved product features to data/processed/ql_product_features.csv


,product_id,p_total_purchases,p_distinct_users,p_avg_add_to_cart,p_reorder_prob
0,1,1852,716,5.801836,0.613391
1,2,90,78,9.888889,0.133333
2,3,277,74,6.415162,0.732852
3,4,329,182,9.507599,0.446809
4,5,15,6,6.466667,0.600000


In [54]:
# Build user-product features from prior only
up_grp = prior.groupby(["user_id", "product_id"])
up_features = pd.DataFrame({
    "user_id": up_grp["user_id"].first(),
    "product_id": up_grp["product_id"].first(),
    "up_times_bought": up_grp["order_id"].count(),
    "up_last_order_number": up_grp["order_number"].max(),
    "up_first_order_number": up_grp["order_number"].min(),
    "up_avg_add_to_cart": up_grp["add_to_cart_order"].mean()
})

# reset index so up_features is column, not index
up_features = up_features.reset_index(drop = True)


In [55]:
# Compute recency
u_last_prior_order = prior.groupby("user_id")["order_number"].max()

# map() to look my max order number for every user_id in up_features
up_features["u_last_prior_order"] = up_features["user_id"].map(u_last_prior_order)

# calculate recency using column subtraction
up_features["up_recency"] = up_features["u_last_prior_order"] - up_features["up_last_order_number"]


In [56]:
# Compute rate in user history
#calculate total items bought per customer
user_total_bought_series = up_features.groupby("user_id")["up_times_bought"].sum()

# map the total sum back onto the up_features table using user_id
up_features["user_total_bought"] = up_features["user_id"].map(user_total_bought_series)

# final rate calculation
up_features["up_rate_in_user_history"] = (
    up_features["up_times_bought"] / up_features["user_total_bought"])

up_features.to_csv(PROCESSED / "ql_user_product_features_prior.csv", index = False)
print("Saved user-product features to data/processed/ql_user_product_features_prior.csv")
up_features.head()

Saved user-product features to data/processed/ql_user_product_features_prior.csv


,user_id,product_id,up_times_bought,up_last_order_number,up_first_order_number,up_avg_add_to_cart,u_last_prior_order,up_recency,user_total_bought,up_rate_in_user_history
0,1,196,10,10,1,1.400000,10,0,59,0.169492
1,1,10258,9,10,2,3.333333,10,0,59,0.152542
2,1,10326,1,5,5,5.000000,10,5,59,0.016949
3,1,12427,10,10,1,3.300000,10,0,59,0.169492
4,1,13032,3,10,2,6.333333,10,0,59,0.050847


In [45]:
# Build train candidates
# op_train contains all products actually bought in train orders
# We merge with orders_train to get user_id for each order
train_labels = op_train.merge(
    orders_train[["order_id", "user_id"]],
    on = "order_id",
    how = "left"
    )

# Assign label=1 for these actual purchases
train_labels["label"] = 1
# Only keep necessary columns for labeling
train_labels = train_labels[["order_id", "user_id", "product_id", "label"]]

For each train order, we want to predict which products the user will buy.
To do this, we consider all products the user has bought in prior orders as candidates.

In [57]:
#user_prior_products: dict mapping user_id to array of product_ids they've bought before.
user_prior_products = prior.groupby("user_id")["product_id"].unique()
candidate_rows = []

# Loop through each train order
for _, order in orders_train.iterrows():
    user_id = order["user_id"]
    order_id = order["order_id"]

# Context features for this order (day of week, hour, days since prior)
context = order[["order_dow", "order_hour_of_day", "days_since_prior_order"]]
# Get all prior products for this user
products = user_prior_products.get(user_id, [])
# For each prior product, create a candidate row
for product_id in products:
    row = {
    "order_id": order_id,
    "user_id": user_id,
    "product_id": product_id,
    "order_dow": context["order_dow"],
    "order_hour_of_day": context["order_hour_of_day"],
    "days_since_prior_order": context["days_since_prior_order"]
    }
candidate_rows.append(row)

In [59]:
# All candidate rows for all train orders
train_candidates = pd.DataFrame(candidate_rows)

# Merge with train_labels: if candidate matches an actual purchase, label=1, else label=0
train_candidates = train_candidates.merge(
train_labels,
on = ["order_id", "user_id", "product_id"],
how = "left")

# Fill missing labels (not actual purchase) with 0
train_candidates["label"] = train_candidates["label"].fillna(0).astype("int8")

# Add user-level features
train_candidates = train_candidates.merge(user_features, on = "user_id", how = "left")
# Add product-level features
train_candidates = train_candidates.merge(product_features, on = "product_id", how = "left")
# Add user-product interaction features
train_candidates = train_candidates.merge(up_features, on = ["user_id", "product_id"], how = "left")

# Print shape and first few rows for inspection
train_candidates.shape
train_candidates.head()

(1, 26)

,order_id,user_id,product_id,order_dow,order_hour_of_day,days_since_prior_order,label,u_total_orders,u_total_items,u_distinct_products,...,p_avg_add_to_cart,p_reorder_prob,up_times_bought,up_last_order_number,up_first_order_number,up_avg_add_to_cart,u_last_prior_order,up_recency,user_total_bought,up_rate_in_user_history
0,272231.0,206209.0,20842,6.0,14.0,30.0,0,13,129,68,...,6.703869,0.690946,1,1,1,11.0,13,12,129,0.007752


In [ ]:
# Build test candidates (similar to train but without labels)
# user_prior_products: dict mapping user_id to array of product_ids they've bought before
test_candidate_rows = []

# Loop through each test order
for _, order in orders_test.iterrows():
    user_id = order["user_id"]
    order_id = order["order_id"]
    # Context features for this order (day of week, hour, days since prior)
    context = order[["order_dow", "order_hour_of_day", "days_since_prior_order"]]
    # Get all prior products for this user
    products = user_prior_products.get(user_id, [])
    # For each prior product, create a candidate row
    for product_id in products:
        row = {
            "order_id": order_id,
            "user_id": user_id,
            "product_id": product_id,
            "order_dow": context["order_dow"],
            "order_hour_of_day": context["order_hour_of_day"],
            "days_since_prior_order": context["days_since_prior_order"]
        }
test_candidate_rows.append(row)
# All candidate rows for all test orders
test_candidates = pd.DataFrame(test_candidate_rows)

# Add user-level features
test_candidates = test_candidates.merge(user_features, on="user_id", how="left")
# Add product-level features
test_candidates = test_candidates.merge(product_features, on="product_id", how="left")
# Add user-product interaction features
test_candidates = test_candidates.merge(up_features, on=["user_id", "product_id"], how="left")

test_candidates.shape
test_candidates.head()


In [60]:
prior.to_csv(PROCESSED / "prior_line_items.csv", index = False)
orders_prior.to_csv(PROCESSED / "orders_prior.csv", index = False)
orders_train.to_csv(PROCESSED / "orders_train.csv", index = False)
orders_test.to_csv(PROCESSED / "orders_test.csv", index = False)
train_candidates.to_csv(PROCESSED / "train_candidates.csv", index = False)
test_candidates.to_csv(PROCESSED / "test_candidates.csv", index = False)

print(f"Saved processed prior line items to {PROCESSED / 'prior_line_items.csv'}")
print("Saved orders_prior.csv, orders_train.csv, orders_test.csv to processed directory.")
print("Saved train_candidates.csv, test_candidates.csv to processed directory.")

NameError: name 'test_candidates' is not defined